# Evaluates the performance of Logistic Regression Classifier with different training sizes of diferrent datasets

In [91]:
import pandas as pd


In [92]:
# GTZAN_dataset = pd.read_csv("../data/hand-crafted/GTZAN_handcrafted_features.csv")

In [93]:
# X = GTZAN_dataset.drop("label", axis=1)
# y = GTZAN_dataset["label"]

In [94]:
# GTZAN_dataset = pd.read_csv("../data/gtzan/features_30_sec.csv")
# GTZAN_dataset.drop(['filename','length'], axis =1, inplace = True)
# X = GTZAN_dataset.drop(['label'], axis=1)
# y = GTZAN_dataset['label']

In [95]:
fma = pd.read_csv('../data/hand-crafted/FMA-medium-handcrafted-label.csv')
fma.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,track_id,subset,genre_top
0,-287.659267,136.153265,-1.044785,46.998524,-4.480115,24.553571,-2.639639,13.259387,7.618672,-2.094758,...,1.495546,1.251119,1.494395,1.862157,214.978746,428.135264,604.615394,135054,small,Experimental
1,-179.123534,112.503843,32.167569,30.893534,3.766597,6.532825,3.143919,3.558495,-9.535322,10.881085,...,4.128862,3.573876,5.408616,3.928263,401.518601,802.062530,880.749257,135040,medium,Folk
2,-99.555557,99.983701,20.608591,21.318870,15.662856,5.197851,-1.226076,0.869772,-8.267710,-1.382700,...,4.880134,5.931302,7.048175,5.393518,517.643590,1036.383325,1104.245792,135336,small,Pop
3,-108.875410,87.945286,-9.532563,-6.515012,-5.952966,-21.799541,-21.075214,-8.817229,-11.907607,-2.482180,...,5.534257,8.000605,6.076923,4.345370,497.781151,995.472194,1187.864241,135337,small,Pop
4,-150.223013,99.786985,18.814489,52.378629,17.220552,15.986250,3.212827,1.292879,-12.196205,2.725307,...,3.980877,5.231651,4.868610,4.262915,401.210943,800.559865,985.653409,135041,medium,Folk


In [96]:
X = fma.drop(['track_id','genre_top','subset'], axis=1)
y = fma['genre_top']

In [97]:
y

0        Experimental
1                Folk
2                 Pop
3                 Pop
4                Folk
             ...     
20562            Rock
20563         Hip-Hop
20564            Rock
20565         Hip-Hop
20566            Rock
Name: genre_top, Length: 20567, dtype: object

In [98]:
X.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276
0,-287.659267,136.153265,-1.044785,46.998524,-4.480115,24.553571,-2.639639,13.259387,7.618672,-2.094758,...,1.606480,1.933978,1.807735,1.495546,1.251119,1.494395,1.862157,214.978746,428.135264,604.615394
1,-179.123534,112.503843,32.167569,30.893534,3.766597,6.532825,3.143919,3.558495,-9.535322,10.881085,...,6.502058,4.859566,3.853364,4.128862,3.573876,5.408616,3.928263,401.518601,802.062530,880.749257
2,-99.555557,99.983701,20.608591,21.318870,15.662856,5.197851,-1.226076,0.869772,-8.267710,-1.382700,...,8.062387,7.598634,5.926319,4.880134,5.931302,7.048175,5.393518,517.643590,1036.383325,1104.245792
3,-108.875410,87.945286,-9.532563,-6.515012,-5.952966,-21.799541,-21.075214,-8.817229,-11.907607,-2.482180,...,8.843665,6.971639,4.917969,5.534257,8.000605,6.076923,4.345370,497.781151,995.472194,1187.864241
4,-150.223013,99.786985,18.814489,52.378629,17.220552,15.986250,3.212827,1.292879,-12.196205,2.725307,...,5.597839,5.322011,5.433737,3.980877,5.231651,4.868610,4.262915,401.210943,800.559865,985.653409


In [99]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import json

In [100]:
n_train = np.logspace(-2, np.log10(0.8), 10)
n_train

array([0.01      , 0.01627251, 0.02647945, 0.04308869, 0.0701161 ,
       0.11409647, 0.18566355, 0.30212113, 0.49162679, 0.8       ])

Arguments:

(X) Matrix of features

(y) Array of targets

(train_size) Size of the trainset, this trian set will be spllited into more partitions  

In [101]:
def performanceOfTrainSizes(X, y,):
    pipeline = Pipeline([
                            ('StandardScaler',  StandardScaler()),
                            ('meu_classificador', LogisticRegression(max_iter=10000))
                        ])

    n_train = np.logspace(-2, np.log10(0.8), 10)
    n_train = [int(n*X.shape[0]) for n in n_train]
    n_train = [n for n in n_train if n > len(set(y))]
    acc_mean = []
    acc_std = []
    acc_mean_overfit = []
    acc_std_overfit = []
    
    for n in n_train:
        this_acc = []
        this_acc_overfit = []
        for k in tqdm(range(50)):
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=n, stratify=y)
            
            # accurracy
            pipeline.fit(X_train,y_train)
            y_pred = pipeline.predict(X_test)
            acc = accuracy_score(y_pred,y_test)
            # acurracy overfit

            y_pred_overfit = pipeline.predict(X_train)
            acc_overfit = accuracy_score(y_pred_overfit,y_train)

            this_acc_overfit.append(acc_overfit)
            this_acc.append(acc)
        
        this_acc_overfit = np.array(this_acc_overfit)
        this_acc = np.array(this_acc)
        acc_mean.append(this_acc.mean())
        acc_std.append(this_acc.std())
        acc_mean_overfit.append(this_acc_overfit.mean())
        acc_std_overfit.append(this_acc_overfit.std())
    return acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train

In [102]:
def saveMetricsStorage(storageJsonPath ,datasetName, acc_mean, acc_mean_overfit, acc_std, acc_std_overfit, n_train,datasetSize):
    with open(storageJsonPath, 'r') as f:
        storage = json.load(f)
    
    storage[datasetName] = {
        'acc_mean': acc_mean,
        'acc_mean_overfit': acc_mean_overfit,
        'acc_std': acc_std,
        'acc_std_overfit': acc_std_overfit,
        'n_train_': n_train,
        'datasetSize': datasetSize
    }
    
    with open(storageJsonPath, 'w') as f:
        json.dump(storage, f)

In [103]:

acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train = performanceOfTrainSizes(X, y)


100%|██████████| 50/50 [1:02:11<00:00, 74.63s/it]


In [104]:
saveMetricsStorage('../data/newMetrics.json', 'FMA-medium-handcrafted', acc_mean, acc_mean_overfit , acc_std, acc_std_overfit ,n_train,len(y))

In [105]:
acc_mean

[0.4681121697279246,
 0.47553897098799,
 0.481570194276582,
 0.49527361414562265,
 0.5199320261437907,
 0.5479973656769661,
 0.5733106454116663,
 0.5916427476661557,
 0.6065952563121653,
 0.6187311618862421]

In [106]:
acc_mean_overfit 

[1.0,
 0.9992215568862275,
 0.9874632352941177,
 0.9341309255079007,
 0.8453259361997225,
 0.7690792838874682,
 0.7211000523834467,
 0.6874585546434895,
 0.666849965384235,
 0.6547134261228955]

In [107]:
acc_std

[0.015483059730846653,
 0.010296252809830786,
 0.010786332865593731,
 0.006405010381847788,
 0.005600566298115206,
 0.004769691671051535,
 0.003332160360505768,
 0.0037064714825993314,
 0.0031872381222978596,
 0.005238920980976349]

In [108]:
acc_std_overfit

[0.0,
 0.0016734357619738996,
 0.006137504008131253,
 0.011573687255542382,
 0.010985899153254864,
 0.008219061803766269,
 0.006528911088023263,
 0.00426931043448937,
 0.00377890432912159,
 0.0018670195621600528]

In [109]:
n_train

[205, 334, 544, 886, 1442, 2346, 3818, 6213, 10111, 16453]